In [1]:
# Gym stuff
import gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv, ForexEnv


# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO

# Technicals
from finta import TA

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# Project Modules
from collections import deque
from config import LOG_PATH, MODEL_PATH
import cryptomanager as cm
from datetime import datetime
from IPython.display import clear_output
import os
import time
from trader import trader_agent

# The class that will do the work of trading
trade_me = trader_agent()

In [ ]:
# Initial DataFrame to train on
df = cm.historical_data_df(days_delta=10000)
df

In [ ]:
# The range that we want to train on each interval
window_size = 100
lower_bound = len(df) - window_size
upper_bound = len(df)
df[lower_bound:upper_bound]

In [ ]:
# Adding technicals to DataFrame

def add_columns_df(df):
    df['OBV'] = TA.OBV(df) # unsure if I want to use this yet or not
    df['EMA'] = TA.EMA(df, 200)
    df['RSI'] = TA.RSI(df)
    df  = df.join(TA.PIVOT_FIB(df))
    df = df.join(TA.MACD(df))
    df.fillna(0, inplace=True)
    return df
    
df = add_columns_df(df)
df.head()

In [ ]:
# Adding metrics to Env

def add_signals(env, df = df):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    keys= []
    for key in df.columns:
        keys.append(key)
    signal_features = env.df.loc[:, [key for key in keys[1:]]].to_numpy()[start:end]
    return prices, signal_features

In [ ]:
class Cypto_Env(StocksEnv):
    _process_data = add_signals
    

In [ ]:
# The initial training Env before we swtich to up-to-date Env

env2 = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
 
# create and train prediction model

model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH, device="cpu") 

name = str(datetime.now().strftime("%Y-%d-%m-%H-%M") )

episodes = 5
step_count = 1000
for i in range (episodes):
    model = model.learn(total_timesteps=step_count, 
    progress_bar=True,
    ) 
    print(i)

In [ ]:
prev_min = 0
print(len(df)-lower_bound,len(df))

In [ ]:
# past_trades = deque(maxlen=60)
# cur_min = datetime.now().minute
# prev_min = 0
# test_env = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
# wait_time = 1
# while True:
#     try:
#         cur_min = datetime.now().minute
#         if cur_min > prev_min or cur_min == 1:
#             cur_min = datetime.now().minute
#             prev_min = cur_min + wait_time - 1
            
#             df = cm.historical_data_df()

#             df = add_columns_df(df)

#             test_env = Cypto_Env(df=df, window_size=window_size, frame_bound=(window_size,upper_bound))
            
#             def return_obs(df):
#                 start = len(df) - window_size
#                 end = len(df)
#                 keys= []
#                 for key in df.columns:
#                     keys.append(key)
#                 signal_features = df.loc[:, [key for key in keys[1:]]].to_numpy()[start:end]
#                 return signal_features
            
#             obs = return_obs(df)
#             action = model.predict(obs)
#             print("made a prediction")
#             print(action)
#             if action == 1 or action[0] == 1:
#                 trade_me.buy_position_at_market("BTC/USD")
#                 print("b") 
#             if action == 0 or action[0] == 0:
#                 trade_me.sell_position_market("BTCUSD")
#                 print("s")
#             past_trades.append([action, ("price: ", df["Low"].iloc[-1]), ("time: ", datetime.now())])
#             for episode in range(episodes):
#                 model = model.learn(total_timesteps=step_count, 
#                 )
#                 # model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
#                 clear_output()
#                 print("past trades: ", list(past_trades))
#     except Exception as e:
#         print(e)
#         for episode in range(episodes):
#             model = model.learn(total_timesteps=step_count,
#              )
#             # model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
#             print("past trades: ", list(past_trades))



In [ ]:
# Save the Model
# model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))

In [13]:
# Get all USD Crypto tradable on Alpaca

def Crypto_USD() -> list:
    output = []
    cryptos = cm.get_cryptos_tradable()
    for crypto in cryptos:
        if crypto.split("/")[1] == "USD":
            output.append(crypto)
    return output

usd = Crypto_USD()

usd.pop(usd.index("BTC/USD"))


23